In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
import bokeh as bk
import datetime as dt

In [ ]:
from os import listdir

In [ ]:
source_dir = './train/'

In [ ]:
files = [ source_dir+file for file in listdir(source_dir) ]

In [30]:
long_shape = (0,0)
long_df = pd.DataFrame()

for i in range( len(files) ):
    df = pd.read_csv( files[i] )
    if df.shape[0] > long_shape[0]:
        long_shape = df.shape
        long_df = df

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [23]:
df_al = pd.read_csv( './train/comet-02-01_4257530.csv' )

In [24]:
df_al

,Cycle,Block device statistics (per device),Scheduler accounting (per CPU),Infiniband usage (default),Infiniband usage (ext),InfiniBand usage (sw),Intel Haswell Processor (HSW) (per core),Caching Agent (CBo) for SNB (HSW) (per socket),Home Agent Unit for SNB (HSW) (per socket),Integrated Memory Controller for SNB (HSW) (per socket),...,intel_rapl,Memory usage (per socket),Network device usage (per device),NFS system usage,NUMA statistics (per socket),Process statistics,SysV shared memory segment usage,Ram-backed filesystem usage (per mount),Dentry/file/inode cache usage,Virtual memory statistics
0,00:04:52,4.008079e+07,1144302.429,2.511115e+08,4.819806e+12,1.204242e+13,2.897580e+14,2.818051e+14,2.954730e+14,2.252228e+14,...,6.011851e+10,1.469495e+10,7240557.522,0.0,7215273.333,3833806.000,0.0,65961986.5,17175.333,5.644557e+07
1,00:14:52,4.085373e+07,1152873.571,2.511115e+08,4.847640e+12,1.211197e+13,2.900108e+14,2.818142e+14,2.958943e+14,2.252239e+14,...,6.014570e+10,1.473150e+10,7275366.478,0.0,7266832.333,3861296.286,0.0,65961986.5,17191.333,5.762767e+07
2,00:24:52,4.181553e+07,1161444.571,2.511115e+08,4.876849e+12,1.218495e+13,2.902631e+14,2.818235e+14,2.963160e+14,2.252250e+14,...,6.017292e+10,1.476773e+10,7426986.174,0.0,7308126.000,3888391.857,0.0,65961986.5,17239.333,5.903775e+07
3,00:34:53,4.259145e+07,1170015.571,2.511115e+08,4.906575e+12,1.225922e+13,2.905152e+14,2.818329e+14,2.967391e+14,2.252262e+14,...,6.020016e+10,1.480427e+10,7462886.174,0.0,7359487.333,3916242.429,0.0,65961986.5,17191.333,6.022244e+07
4,00:44:53,4.303399e+07,1178586.857,2.511115e+08,4.937391e+12,1.233621e+13,2.907669e+14,2.818425e+14,2.971618e+14,2.252274e+14,...,6.157171e+10,1.484071e+10,7496620.000,0.0,7411319.333,3941879.714,0.0,65961986.5,17191.333,6.090396e+07
5,00:54:53,4.309994e+07,1187157.714,2.511115e+08,4.970377e+12,1.241863e+13,2.910178e+14,2.818525e+14,2.975841e+14,2.252287e+14,...,6.159904e+10,1.488330e+10,7543498.391,0.0,7468152.333,3971321.429,0.0,65961986.5,17191.333,6.100965e+07
6,01:04:53,4.317929e+07,1195728.857,2.511115e+08,5.007388e+12,1.251110e+13,2.912674e+14,2.818632e+14,2.980062e+14,2.252302e+14,...,6.162648e+10,1.493197e+10,7575358.652,0.0,7529380.333,3999906.429,0.0,65961986.5,17223.333,6.112566e+07
7,01:49:39,0.000000e+00,0.000,0.000000e+00,1.000000e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,8.574113e+09,0.000,0.0,0.000,555.286,0.0,0.5,6618.333,0.000000e+00
8,01:59:39,7.519301e+06,8570.571,6.277121e+06,3.604177e+10,9.005105e+10,2.474460e+11,1.182967e+10,4.257571e+11,1.280628e+09,...,2.713098e+07,8.684008e+09,168193.304,0.0,189322.667,70146.571,0.0,65961986.5,9384.000,7.897411e+06
9,02:09:39,7.585613e+06,17141.857,1.317143e+07,7.562150e+10,1.889418e+11,4.961548e+11,2.464110e+10,8.435428e+11,2.667320e+09,...,1.398807e+09,8.726615e+09,207672.348,0.0,241609.333,98090.571,0.0,65961986.5,9377.667,8.014086e+06
